In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, FunctionTransformer   
from sklearn.metrics import confusion_matrix, classification_report, log_loss

from sklearn.model_selection import StratifiedKFold

from collections import Counter
from tqdm.notebook import tqdm

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv('test.csv')

In [3]:
class2idx = {
    'Class_1':0,
    'Class_2':1,
    'Class_3':2,
    'Class_4':3,
    'Class_5':4,
    'Class_6':5,
    'Class_7':6,
    'Class_8':7,
    'Class_9':8,
}

idx2class = {v: k for k, v in class2idx.items()}

train['target'].replace(class2idx, inplace=True)

In [4]:
X = np.array(train.iloc[:, 1:94])
y = np.array(train.iloc[:, -1])

In [5]:
X_final_test = test.iloc[:, 1:94]

In [6]:
transformer = FunctionTransformer(np.log1p, validate=True)

In [7]:
class ClassifierDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [8]:
### Training set on full data
y_full = y
X_full = transformer.fit_transform(X)
### Full dataset
full_dataset = ClassifierDataset(torch.from_numpy(X_full).float(), torch.from_numpy(y_full).long())
### Test dataset
X_final_test = transformer.fit_transform(X_final_test)

In [9]:
class ClassifierDataset_test(Dataset):
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)

test_dataset = ClassifierDataset_test(torch.from_numpy(X_final_test).float())

In [10]:
EPOCHS = 35
batch_size = 128 #128
LEARNING_RATE = 0.03923132744309443
NUM_FEATURES = 93 #len(X.columns)
NUM_CLASSES = 9
MOMENT = 0.5158575830025897

In [50]:
bag_n = 25
NN_preds_list = []
for bag in tqdm(range(1,bag_n+1)):
    test_loader = DataLoader(dataset=test_dataset,
                              batch_size=batch_size
    )
    ### Only for training on full dataset
    train_loader = DataLoader(dataset=full_dataset,
                              batch_size=batch_size,
                              shuffle = True
    )

    class MulticlassClassification(nn.Module):
        def __init__(self, num_feature, num_class):
            super(MulticlassClassification, self).__init__()

            self.layer_1 = nn.Linear(num_feature, 505)
            self.layer_2 = nn.Linear(505, 152)
            self.layer_out = nn.Linear(152, num_class) 

            self.relu = nn.ReLU()
            self.dropout1 = nn.Dropout(p=0.38157117252594575)
            self.dropout2 = nn.Dropout(p=0.20144716052206688)
            self.batchnorm1 = nn.BatchNorm1d(505)
            self.batchnorm2 = nn.BatchNorm1d(152)

        def forward(self, x):
            x = self.layer_1(x)
            x = self.batchnorm1(x)
            x = self.relu(x)
            x = self.dropout1(x)

            x = self.layer_2(x)
            x = self.batchnorm2(x)
            x = self.relu(x)
            x = self.dropout2(x)

            x = self.layer_out(x)

            return x

    model = MulticlassClassification(num_feature = NUM_FEATURES, num_class=NUM_CLASSES)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum = MOMENT)

    def multi_acc(y_pred, y_test):
        y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
        _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    

        correct_pred = (y_pred_tags == y_test).float()
        acc = correct_pred.sum() / len(correct_pred)
        acc = acc * 100

        return acc

    def save_pred(y_pred, y_test):
        y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
        _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    

        return y_pred_tags, y_test

    accuracy_stats = {
        'train': [],
        "val": []
    }
    loss_stats = {
        'train': [],
        "val": []
    }

    print("Begin training.")
    for e in tqdm(range(1, EPOCHS+1)):

        # TRAINING
        train_epoch_loss = 0
        train_epoch_acc = 0

        model.train()
        for X_train_batch, y_train_batch in train_loader:
            optimizer.zero_grad()

            y_train_pred = model(X_train_batch)

            train_loss = criterion(y_train_pred, y_train_batch)
            train_acc = multi_acc(y_train_pred, y_train_batch)

            train_loss.backward()
            optimizer.step()

            train_epoch_loss += train_loss.item()
            train_epoch_acc += train_acc.item()

        loss_stats['train'].append(train_epoch_loss/len(train_loader))
        accuracy_stats['train'].append(train_epoch_acc/len(train_loader))                       

        print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.3f} | Train Acc: {train_epoch_acc/len(train_loader):.3f}')
        
    ### Prediction
    y_pred_list = []
    with torch.no_grad():
        model.eval()
        for X_batch, y_batch in test_loader: #in test_loader:
            y_test_pred = model(X_batch)
            y_test_pred_prob = nn.functional.softmax(y_test_pred, dim = 1)
            y_pred_list.append(y_test_pred_prob.numpy())

    test_prediction = np.array(y_pred_list)
    pred_df = pd.DataFrame(pd.DataFrame(np.concatenate(test_prediction)))
    NN_prediction = pred_df.to_numpy()
    NN_preds_list.append(NN_prediction)

  0%|          | 0/25 [00:00<?, ?it/s]

Begin training.


  0%|          | 0/35 [00:00<?, ?it/s]

Epoch 001: | Train Loss: 0.749 | Train Acc: 72.711
Epoch 002: | Train Loss: 0.620 | Train Acc: 76.146
Epoch 003: | Train Loss: 0.591 | Train Acc: 77.316
Epoch 004: | Train Loss: 0.573 | Train Acc: 77.937
Epoch 005: | Train Loss: 0.555 | Train Acc: 78.453
Epoch 006: | Train Loss: 0.545 | Train Acc: 78.706
Epoch 007: | Train Loss: 0.535 | Train Acc: 78.987
Epoch 008: | Train Loss: 0.527 | Train Acc: 79.521
Epoch 009: | Train Loss: 0.519 | Train Acc: 79.602
Epoch 010: | Train Loss: 0.513 | Train Acc: 79.814
Epoch 011: | Train Loss: 0.508 | Train Acc: 80.110
Epoch 012: | Train Loss: 0.501 | Train Acc: 80.290
Epoch 013: | Train Loss: 0.494 | Train Acc: 80.319
Epoch 014: | Train Loss: 0.489 | Train Acc: 80.615
Epoch 015: | Train Loss: 0.486 | Train Acc: 80.825
Epoch 016: | Train Loss: 0.482 | Train Acc: 81.004
Epoch 017: | Train Loss: 0.478 | Train Acc: 80.940
Epoch 018: | Train Loss: 0.475 | Train Acc: 81.156
Epoch 019: | Train Loss: 0.472 | Train Acc: 81.162
Epoch 020: | Train Loss: 0.466 

<ipython-input-50-75860f155e9b>:108: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  test_prediction = np.array(y_pred_list)


...

Epoch 001: | Train Loss: 0.753 | Train Acc: 72.468
Epoch 002: | Train Loss: 0.621 | Train Acc: 76.385
Epoch 003: | Train Loss: 0.595 | Train Acc: 77.015
Epoch 004: | Train Loss: 0.573 | Train Acc: 77.780
Epoch 005: | Train Loss: 0.559 | Train Acc: 78.216
Epoch 006: | Train Loss: 0.546 | Train Acc: 78.668
Epoch 007: | Train Loss: 0.536 | Train Acc: 79.015
Epoch 008: | Train Loss: 0.529 | Train Acc: 79.077
Epoch 009: | Train Loss: 0.520 | Train Acc: 79.516
Epoch 010: | Train Loss: 0.516 | Train Acc: 79.675
Epoch 011: | Train Loss: 0.509 | Train Acc: 79.879
Epoch 012: | Train Loss: 0.505 | Train Acc: 80.145
Epoch 013: | Train Loss: 0.497 | Train Acc: 80.521
Epoch 014: | Train Loss: 0.494 | Train Acc: 80.432
Epoch 015: | Train Loss: 0.488 | Train Acc: 80.706
Epoch 016: | Train Loss: 0.484 | Train Acc: 80.746
Epoch 017: | Train Loss: 0.480 | Train Acc: 81.093
Epoch 018: | Train Loss: 0.475 | Train Acc: 81.127
Epoch 019: | Train Loss: 0.474 | Train Acc: 81.200
Epoch 020: | Train Loss: 0.472 

  0%|          | 0/35 [00:00<?, ?it/s]

Epoch 001: | Train Loss: 0.752 | Train Acc: 72.867
Epoch 002: | Train Loss: 0.621 | Train Acc: 76.414
Epoch 003: | Train Loss: 0.591 | Train Acc: 77.125
Epoch 004: | Train Loss: 0.570 | Train Acc: 77.845
Epoch 005: | Train Loss: 0.556 | Train Acc: 78.418
Epoch 006: | Train Loss: 0.544 | Train Acc: 78.782
Epoch 007: | Train Loss: 0.539 | Train Acc: 78.885
Epoch 008: | Train Loss: 0.529 | Train Acc: 79.282
Epoch 009: | Train Loss: 0.520 | Train Acc: 79.549
Epoch 010: | Train Loss: 0.513 | Train Acc: 79.934
Epoch 011: | Train Loss: 0.508 | Train Acc: 79.926
Epoch 012: | Train Loss: 0.504 | Train Acc: 80.001
Epoch 013: | Train Loss: 0.494 | Train Acc: 80.345
Epoch 014: | Train Loss: 0.493 | Train Acc: 80.580
Epoch 015: | Train Loss: 0.488 | Train Acc: 80.578
Epoch 016: | Train Loss: 0.482 | Train Acc: 80.829
Epoch 017: | Train Loss: 0.476 | Train Acc: 81.075
Epoch 018: | Train Loss: 0.475 | Train Acc: 81.184
Epoch 019: | Train Loss: 0.473 | Train Acc: 81.027
Epoch 020: | Train Loss: 0.469 

  0%|          | 0/35 [00:00<?, ?it/s]

Epoch 001: | Train Loss: 0.755 | Train Acc: 72.468
Epoch 002: | Train Loss: 0.622 | Train Acc: 76.168
Epoch 003: | Train Loss: 0.592 | Train Acc: 77.056
Epoch 004: | Train Loss: 0.572 | Train Acc: 77.880
Epoch 005: | Train Loss: 0.558 | Train Acc: 78.463
Epoch 006: | Train Loss: 0.548 | Train Acc: 78.659
Epoch 007: | Train Loss: 0.537 | Train Acc: 78.858
Epoch 008: | Train Loss: 0.530 | Train Acc: 79.269
Epoch 009: | Train Loss: 0.519 | Train Acc: 79.577
Epoch 010: | Train Loss: 0.517 | Train Acc: 79.750
Epoch 011: | Train Loss: 0.510 | Train Acc: 80.002
Epoch 012: | Train Loss: 0.503 | Train Acc: 79.956
Epoch 013: | Train Loss: 0.498 | Train Acc: 80.332
Epoch 014: | Train Loss: 0.494 | Train Acc: 80.341
Epoch 015: | Train Loss: 0.487 | Train Acc: 80.745
Epoch 016: | Train Loss: 0.483 | Train Acc: 80.773
Epoch 017: | Train Loss: 0.480 | Train Acc: 80.839
Epoch 018: | Train Loss: 0.475 | Train Acc: 81.119
Epoch 019: | Train Loss: 0.470 | Train Acc: 81.426
Epoch 020: | Train Loss: 0.469 

### Bagging

In [26]:
bag_nn = 0
for i in range(len(NN_preds_list)):
        bag_nn += NN_preds_list[i]
        
bag_nn = bag_nn/len(NN_preds_list)

In [33]:
bag_nn = pd.DataFrame(bag_nn)
bag_nn.columns = ['Class_1','Class_2','Class_3','Class_4','Class_5','Class_6','Class_7','Class_8','Class_9']
bag_nn.insert(0,"id", test['id'])

,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,1,2.266976e-05,1.782722e-01,1.723621e-01,6.464968e-01,4.904274e-07,2.575393e-06,2.840546e-03,1.787647e-06,9.686939e-07
1,2,3.654512e-05,4.140747e-04,2.709027e-05,8.081172e-06,7.321881e-05,1.786772e-01,2.347893e-05,8.205765e-01,1.636986e-04
2,3,2.924198e-09,1.114675e-10,7.176854e-12,1.114731e-09,6.597482e-11,9.999980e-01,5.494100e-09,2.067520e-06,1.159001e-09
3,4,1.615727e-06,4.996934e-01,4.952756e-01,4.997120e-03,4.026070e-07,6.646044e-08,7.230842e-06,1.755419e-07,2.451029e-05
4,5,2.630280e-01,4.056917e-06,8.249455e-07,3.784378e-07,4.295207e-07,4.266151e-04,1.251044e-04,2.323879e-02,7.131759e-01
...,...,...,...,...,...,...,...,...,...,...
144363,144364,2.194854e-01,3.312969e-03,5.323167e-04,8.635801e-03,3.967461e-04,6.970814e-01,2.403968e-02,1.638010e-02,3.013544e-02
144364,144365,3.430317e-04,3.178454e-01,5.453628e-01,9.104673e-02,1.552901e-04,2.503050e-04,4.494089e-02,3.115381e-05,2.445987e-05
144365,144366,9.126494e-06,5.553191e-01,3.363813e-01,1.038502e-01,3.504857e-06,1.051230e-04,4.309203e-03,1.835412e-05,4.035709e-06
144366,144367,9.332928e-05,4.642993e-01,3.020846e-02,5.050222e-01,2.225977e-05,1.080069e-05,3.318048e-04,7.976754e-06,3.871290e-06


In [34]:
bag_nn.to_csv(f'~/Desktop/Data Science Projects/Otto Group /NN Predictions/NN_bag{len(NN_preds_list)}.csv',
               header = True, index = False)

#### Joining the 5 NN i made previously to this one

In [40]:
bag5 = pd.read_csv('~/Desktop/Data Science Projects/Otto Group /NN Predictions/NN_bag5.csv')
bag5 = bag5.iloc[:,1:].to_numpy()
bag5 = bag5*5

In [42]:
bag20 = pd.read_csv('~/Desktop/Data Science Projects/Otto Group /NN Predictions/NN_bag20.csv')
bag20 = bag20.iloc[:,1:].to_numpy()
bag20 = bag20*20

In [45]:
bag25 = (bag5+bag20)/25
bag25 = pd.DataFrame(bag25)
bag25.columns = ['Class_1','Class_2','Class_3','Class_4','Class_5','Class_6','Class_7','Class_8','Class_9']
bag25.insert(0,"id", test['id'])

In [47]:
bag25.to_csv(f'~/Desktop/Data Science Projects/Otto Group /NN Predictions/NN_bag{25}.csv',
               header = True, index = False)